In [104]:
!pip install -e .

Obtaining file:///Users/matthewdeane/Documents/Data%20Science/python/_projects/py-tidymodels/_md
ERROR: file:///Users/matthewdeane/Documents/Data%20Science/python/_projects/py-tidymodels/_md does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [105]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# py-tidymodels imports
from py_workflows import workflow
from py_parsnip import linear_reg
from py_rsample import vfold_cv, initial_split, training, testing
from py_yardstick import metric_set, rmse, mae, r_squared, smape
from py_workflowsets import WorkflowSet

from py_recipes import recipe
from py_recipes.selectors import all_numeric, all_nominal, starts_with, ends_with

# Set random seed for reproducibility
np.random.seed(42)

# Import Data

In [106]:
# import raw data
raw_data = pd.read_csv('__data/preem.csv')

In [107]:
df = raw_data

# set date as index
#df = raw_data.set_index('date')

# convert date to datetime
df['date'] = pd.to_datetime(df['date'])
display(df)

,date,mean_med_diesel_crack_input1_trade_month_lag2,mean_nwe_hsfo_crack_trade_month_lag1,mean_nwe_lsfo_crack_trade_month,mean_nwe_ulsfo_crack_trade_month lag3,mean_sing_gasoline_vs_vlsfo_trade_month,mean_sing_vlsfo_crack_trade_month_lag3,new_sweet_sr_margin,target,totaltar
0,2020-04-01,-56.18,-11.43,-3.07,28.58,-16.09,26.91,0.47,137.65,0.00
1,2020-05-01,-42.36,-9.17,3.89,22.64,-8.75,20.36,0.57,113.53,0.00
2,2020-06-01,-30.21,-8.58,-3.44,13.43,-4.28,11.32,6.61,43.31,0.00
3,2020-07-01,-28.86,-6.86,-2.71,10.74,-3.20,8.53,-1.55,79.77,0.00
4,2020-08-01,-38.80,-5.86,-2.63,4.58,-5.35,6.26,-2.70,48.27,0.00
5,2020-09-01,-41.76,-5.62,-2.08,7.69,-3.26,6.15,-2.10,98.01,0.00
6,2020-10-01,-43.99,-5.10,0.16,8.09,-4.31,5.80,-3.34,49.77,0.00
7,2020-11-01,-42.94,-5.43,1.80,7.86,-7.40,6.99,-2.53,35.48,0.00
8,2020-12-01,-41.55,-3.39,0.94,6.68,-7.65,6.22,0.00,63.98,0.00
9,2021-01-01,-41.18,-6.03,0.63,8.67,-7.67,8.22,-0.05,109.76,0.00


# py-rsample

In [108]:
from py_rsample import initial_time_split, time_series_cv

In [109]:
# Create train/test split
split = initial_split(df, prop=0.75, seed=123)
train_data = training(split)
test_data = testing(split)

print(f"Training set: {train_data.shape[0]} rows")
print(f"Test set: {test_data.shape[0]} rows")

Training set: 42 rows
Test set: 15 rows


In [110]:
# Example: Train on 2020, test on first half of 2021
split_absolute = initial_time_split(
    df,
    date_column="date",
    train_start="2022-01-01",
    train_end="2023-12-01",
    test_start="2024-01-01",
    test_end="2024-06-01"
)

train_abs = split_absolute.training()
test_abs = split_absolute.testing()

print(f"Training: {len(train_abs)} days from {train_abs['date'].min()} to {train_abs['date'].max()}")
print(f"Testing:  {len(test_abs)} days from {test_abs['date'].min()} to {test_abs['date'].max()}")

# bug - thinks its days not months (monthly data)

Training: 24 days from 2022-01-01 00:00:00 to 2023-12-01 00:00:00
Testing:  6 days from 2024-01-01 00:00:00 to 2024-06-01 00:00:00


In [111]:
# Example 1: Train on first 18 months, test on last 6 months
split_relative1 = initial_time_split(
    df,
    date_column="date",
    train_start="start",
    train_end="start + 18 months",
    test_start="end - 6 months",
    test_end="end"
)

train_rel1 = split_relative1.training()
test_rel1 = split_relative1.testing()

print("Example 1: First 18 months vs last 6 months")
print(f"Training: {len(train_rel1)} days from {train_rel1['date'].min()} to {train_rel1['date'].max()}")
print(f"Testing:  {len(test_rel1)} days from {test_rel1['date'].min()} to {test_rel1['date'].max()}")
print()

# bug - thinks its days not months (monthly data)

Example 1: First 18 months vs last 6 months
Training: 18 days from 2020-04-01 00:00:00 to 2021-09-01 00:00:00
Testing:  6 days from 2024-07-01 00:00:00 to 2024-12-01 00:00:00



In [112]:
# Train on specific year (2020), test on last 3 months of available data
split_mixed = initial_time_split(
    df,
    date_column="date",
    train_start="start",
    train_end="2023-12-01",
    test_start="end - 12 months",
    test_end="end"
)

train_mix = split_mixed.training()
test_mix = split_mixed.testing()

print("Mixed: Train on 2020, test on last 90 days of data")
print(f"Training: {len(train_mix)} days from {train_mix['date'].min()} to {train_mix['date'].max()}")
print(f"Testing:  {len(test_mix)} days from {test_mix['date'].min()} to {test_mix['date'].max()}")

# bug - thinks its days not months (monthly data)

Mixed: Train on 2020, test on last 90 days of data
Training: 45 days from 2020-04-01 00:00:00 to 2023-12-01 00:00:00
Testing:  12 days from 2024-01-01 00:00:00 to 2024-12-01 00:00:00


# 1. Single Model Fitting

In [113]:
FORMULA_STR = "target ~ totaltar +mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1"

## 1.1 - Prophet

In [114]:
# SINGLE MODEL fitting

from py_parsnip import prophet_reg, arima_reg, rand_forest, linear_reg

# Create Prophet specification
# Prophet will automatically detect weekly seasonality in our data
spec_prophet = prophet_reg(
    n_changepoints=25,  # Number of potential trend changes
    changepoint_prior_scale=0.05,  # Flexibility of trend
    seasonality_prior_scale=10.0   # Flexibility of seasonality
)

print(spec_prophet)

# # Fit Prophet - FAILING
fit_prophet = spec_prophet.fit(train_data, FORMULA_STR)
print("Prophet model fitted!")

# Evaluate and extract outputs
fit_prophet = fit_prophet.evaluate(test_data)
outputs_prophet, coefs_prophet, stats_prophet = fit_prophet.extract_outputs()

display(outputs_prophet)
display(coefs_prophet)
display(stats_prophet)

ModelSpec(model_type='prophet_reg', engine='prophet', mode='regression', args={'growth': 'linear', 'changepoint_prior_scale': 0.05, 'seasonality_prior_scale': 10.0, 'seasonality_mode': 'additive', 'n_changepoints': 25, 'changepoint_range': 0.8, 'yearly_seasonality': 'auto', 'weekly_seasonality': 'auto', 'daily_seasonality': 'auto'}, date_col=None)
Prophet model fitted!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,112.812329,137.65,24.837671,train,prophet_reg,,global
1,2020-05-01,113.53,115.589899,113.53,-2.059899,train,prophet_reg,,global
2,2020-06-01,43.31,66.349488,43.31,-23.039488,train,prophet_reg,,global
3,2020-07-01,79.77,70.154397,79.77,9.615603,train,prophet_reg,,global
4,2020-08-01,48.27,64.104879,48.27,-15.834879,train,prophet_reg,,global
5,2020-09-01,98.01,118.531915,98.01,-20.521915,train,prophet_reg,,global
6,2020-10-01,49.77,79.545138,49.77,-29.775138,train,prophet_reg,,global
7,2020-11-01,35.48,58.856986,35.48,-23.376986,train,prophet_reg,,global
8,2020-12-01,63.98,99.834320,63.98,-35.854320,train,prophet_reg,,global
9,2021-01-01,109.76,115.407468,109.76,-5.647468,train,prophet_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,growth,linear,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
1,changepoint_prior_scale,0.05,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
2,seasonality_prior_scale,10.0,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
3,seasonality_mode,additive,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
4,n_changepoints,25,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,21.537087,train,prophet_reg,,global
1,mae,17.262224,train,prophet_reg,,global
2,mape,19.910901,train,prophet_reg,,global
3,smape,18.557042,train,prophet_reg,,global
4,r_squared,0.626847,train,prophet_reg,,global
5,mda,80.487805,train,prophet_reg,,global
6,rmse,45.21693,test,prophet_reg,,global
7,mae,38.048444,test,prophet_reg,,global
8,mape,29.854629,test,prophet_reg,,global
9,smape,32.221975,test,prophet_reg,,global


## 1.2 - ARIMA

In [115]:
# Create ARIMA specification
spec_arima = arima_reg(
    seasonal_period=7,  # Weekly seasonality
    non_seasonal_ar=1,
    non_seasonal_differences=1,
    non_seasonal_ma=1,
    seasonal_ar=1,
    seasonal_differences=0,
    seasonal_ma=1
)

print(spec_arima)

# Fit ARIMA
fit_arima = spec_arima.fit(train_data, FORMULA_STR)
print("ARIMA model fitted!")

# Predict on test data
pred_arima = fit_arima.predict(test_data)

# Evaluate and extract outputs
fit_arima = fit_arima.evaluate(test_data)
outputs_arima, coefs_arima, stats_arima = fit_arima.extract_outputs()

display(outputs_arima)
display(coefs_arima)
display(stats_arima)

ModelSpec(model_type='arima_reg', engine='statsmodels', mode='regression', args={'seasonal_period': 7, 'non_seasonal_ar': 1, 'non_seasonal_differences': 1, 'non_seasonal_ma': 1, 'seasonal_ar': 1, 'seasonal_differences': 0, 'seasonal_ma': 1}, date_col=None)
ARIMA model fitted!


/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,5.250682,137.65,132.399318,train,arima_reg,,global
1,2020-05-01,113.53,150.527218,113.53,-36.997218,train,arima_reg,,global
2,2020-06-01,43.31,107.848484,43.31,-64.538484,train,arima_reg,,global
3,2020-07-01,79.77,28.240591,79.77,51.529409,train,arima_reg,,global
4,2020-08-01,48.27,88.007526,48.27,-39.737526,train,arima_reg,,global
5,2020-09-01,98.01,41.646030,98.01,56.363970,train,arima_reg,,global
6,2020-10-01,49.77,108.570545,49.77,-58.800545,train,arima_reg,,global
7,2020-11-01,35.48,40.260428,35.48,-4.780428,train,arima_reg,,global
8,2020-12-01,63.98,31.617338,63.98,32.362662,train,arima_reg,,global
9,2021-01-01,109.76,69.716422,109.76,40.043578,train,arima_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,totaltar,-0.301816,0.195180,-1.546350e+00,0.122020,-0.684362,0.080729,NaN,arima_reg,,global
1,mean_med_diesel_crack_input1_trade_month_lag2,-0.060808,0.097095,-6.262692e-01,0.531138,-0.251110,0.129495,NaN,arima_reg,,global
2,mean_nwe_hsfo_crack_trade_month_lag1,-0.160499,0.361724,-4.437072e-01,0.657254,-0.869465,0.548466,NaN,arima_reg,,global
3,ar.L1,0.212695,0.211311,1.006549e+00,0.314151,-0.201467,0.626856,NaN,arima_reg,,global
4,ma.L1,-1.000002,0.259247,-3.857324e+00,0.000115,-1.508117,-0.491886,NaN,arima_reg,,global
5,ar.S.L7,0.011709,0.346587,3.378505e-02,0.973049,-0.667588,0.691007,NaN,arima_reg,,global
6,ma.S.L7,0.314804,0.570340,5.519592e-01,0.580976,-0.803041,1.432650,NaN,arima_reg,,global
7,sigma2,727.308723,0.000356,2.040401e+06,0.000000,727.308024,727.309421,NaN,arima_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,38.152458,train,arima_reg,,global
1,mae,29.125517,train,arima_reg,,global
2,mape,34.139536,train,arima_reg,,global
3,smape,33.933949,train,arima_reg,,global
4,r_squared,-0.171003,train,arima_reg,,global
5,mda,53.658537,train,arima_reg,,global
6,rmse,37.637993,test,arima_reg,,global
7,mae,28.97517,test,arima_reg,,global
8,mape,21.216024,test,arima_reg,,global
9,smape,25.068372,test,arima_reg,,global


## 1.3 - OLS - Statsmodels

In [116]:
from py_parsnip import linear_reg

# Create specification with statsmodels engine
spec_sm = linear_reg().set_engine("statsmodels")

# Fit model
fit_sm = spec_sm.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_sm = fit_sm.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_sm, coefs_sm, stats_sm = fit_sm.extract_outputs()

display(outputs_sm)
display(coefs_sm)
display(stats_sm)

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,103.031150,137.65,34.618850,train,linear_reg,,global
1,2020-05-01,113.53,100.680966,113.53,12.849034,train,linear_reg,,global
2,2020-06-01,43.31,99.272716,43.31,-55.962716,train,linear_reg,,global
3,2020-07-01,79.77,98.337003,79.77,-18.567003,train,linear_reg,,global
4,2020-08-01,48.27,98.790761,48.27,-50.520761,train,linear_reg,,global
5,2020-09-01,98.01,98.953101,98.01,-0.943101,train,linear_reg,,global
6,2020-10-01,49.77,98.915648,49.77,-49.145648,train,linear_reg,,global
7,2020-11-01,35.48,98.973392,35.48,-63.493392,train,linear_reg,,global
8,2020-12-01,63.98,97.883238,63.98,-33.903238,train,linear_reg,,global
9,2021-01-01,109.76,99.092251,109.76,10.667749,train,linear_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,92.420998,8.352429,11.065165,1.903858e-13,75.512390,109.329606,NaN,linear_reg,,global
1,totaltar,-0.274647,0.110660,-2.481899,1.760500e-02,-0.498666,-0.050627,1.004942,linear_reg,,global
2,mean_med_diesel_crack_input1_trade_month_lag2,-0.093034,0.097941,-0.949898,3.481667e-01,-0.291305,0.105237,1.009927,linear_reg,,global
3,mean_nwe_hsfo_crack_trade_month_lag1,-0.470998,0.420807,-1.119275,2.700480e-01,-1.322877,0.380880,1.005225,linear_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,31.772848,train,linear_reg,,global
1,mae,25.035409,train,linear_reg,,global
2,mape,31.042183,train,linear_reg,,global
3,smape,26.008852,train,linear_reg,,global
4,r_squared,0.18787,train,linear_reg,,global
5,mda,58.536585,train,linear_reg,,global
6,adj_r_squared,0.100073,train,linear_reg,,global
7,rmse,39.604179,test,linear_reg,,global
8,mae,31.330025,test,linear_reg,,global
9,mape,23.043006,test,linear_reg,,global


## 1.4 - OLS - sklearn

In [117]:
# sklearn engine (default)
spec_sk = linear_reg()
fit_sk = spec_sk.fit(train_data, FORMULA_STR)
fit_sk = fit_sk.evaluate(test_data)

print("sklearn model fitted successfully!")

outputs_sk, coefs_sk, stats_sk = fit_sk.extract_outputs()
display(outputs_sk)
display(coefs_sk)
display(stats_sk)

sklearn model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,103.031150,137.65,34.618850,train,linear_reg,,global
1,2020-05-01,113.53,100.680966,113.53,12.849034,train,linear_reg,,global
2,2020-06-01,43.31,99.272716,43.31,-55.962716,train,linear_reg,,global
3,2020-07-01,79.77,98.337003,79.77,-18.567003,train,linear_reg,,global
4,2020-08-01,48.27,98.790761,48.27,-50.520761,train,linear_reg,,global
5,2020-09-01,98.01,98.953101,98.01,-0.943101,train,linear_reg,,global
6,2020-10-01,49.77,98.915648,49.77,-49.145648,train,linear_reg,,global
7,2020-11-01,35.48,98.973392,35.48,-63.493392,train,linear_reg,,global
8,2020-12-01,63.98,97.883238,63.98,-33.903238,train,linear_reg,,global
9,2021-01-01,109.76,99.092251,109.76,10.667749,train,linear_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,92.420998,8.352429,11.065165,1.905143e-13,75.512390,109.329606,NaN,linear_reg,,global
1,totaltar,-0.274647,0.110660,-2.481899,1.760500e-02,-0.498666,-0.050627,1.004942,linear_reg,,global
2,mean_med_diesel_crack_input1_trade_month_lag2,-0.093034,0.097941,-0.949898,3.481667e-01,-0.291305,0.105237,1.009927,linear_reg,,global
3,mean_nwe_hsfo_crack_trade_month_lag1,-0.470998,0.420807,-1.119275,2.700480e-01,-1.322877,0.380880,1.005225,linear_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,31.772848,train,linear_reg,,global
1,mae,25.035409,train,linear_reg,,global
2,mape,31.042183,train,linear_reg,,global
3,smape,26.008852,train,linear_reg,,global
4,r_squared,0.18787,train,linear_reg,,global
5,mda,58.536585,train,linear_reg,,global
6,adj_r_squared,0.123755,train,linear_reg,,global
7,rmse,39.604179,test,linear_reg,,global
8,mae,31.330025,test,linear_reg,,global
9,mape,23.043006,test,linear_reg,,global


In [118]:
from py_parsnip import linear_reg

# Create specification with statsmodels engine
spec_elasticnet = linear_reg(penalty = 0.1, mixture = 0.5).set_engine("sklearn")

# Fit model
fit_elasticnet = spec_elasticnet.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_elasticnet = fit_elasticnet.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_elasticnet, coefs_elasticnet, stats_elasticnet = fit_elasticnet.extract_outputs()

display(outputs_elasticnet)
display(coefs_elasticnet)
display(stats_elasticnet)

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,103.030212,137.65,34.619788,train,linear_reg,,global
1,2020-05-01,113.53,100.681260,113.53,12.848740,train,linear_reg,,global
2,2020-06-01,43.31,99.273415,43.31,-55.963415,train,linear_reg,,global
3,2020-07-01,79.77,98.338550,79.77,-18.568550,train,linear_reg,,global
4,2020-08-01,48.27,98.792697,48.27,-50.522697,train,linear_reg,,global
5,2020-09-01,98.01,98.955125,98.01,-0.945125,train,linear_reg,,global
6,2020-10-01,49.77,98.917902,49.77,-49.147902,train,linear_reg,,global
7,2020-11-01,35.48,98.975496,35.48,-63.495496,train,linear_reg,,global
8,2020-12-01,63.98,97.886346,63.98,-33.906346,train,linear_reg,,global
9,2021-01-01,109.76,99.094081,109.76,10.665919,train,linear_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,92.426157,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
1,totaltar,-0.274617,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
2,mean_med_diesel_crack_input1_trade_month_lag2,-0.093024,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
3,mean_nwe_hsfo_crack_trade_month_lag1,-0.470513,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,31.772848,train,linear_reg,,global
1,mae,25.035688,train,linear_reg,,global
2,mape,31.043648,train,linear_reg,,global
3,smape,26.010013,train,linear_reg,,global
4,r_squared,0.18787,train,linear_reg,,global
5,mda,58.536585,train,linear_reg,,global
6,adj_r_squared,0.123755,train,linear_reg,,global
7,rmse,39.601142,test,linear_reg,,global
8,mae,31.327756,test,linear_reg,,global
9,mape,23.041318,test,linear_reg,,global


In [119]:
from py_parsnip import linear_reg

# Create specification with statsmodels engine
spec_ridge  = linear_reg(penalty = 0.1, mixture = 0).set_engine("sklearn")

# Fit model
fit_ridge = spec_ridge.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_ridge = fit_ridge.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_ridge, coefs_ridge, stats_ridge = fit_ridge.extract_outputs()

display(outputs_ridge)
display(coefs_ridge)
display(stats_ridge)

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,103.031138,137.65,34.618862,train,linear_reg,,global
1,2020-05-01,113.53,100.680970,113.53,12.849030,train,linear_reg,,global
2,2020-06-01,43.31,99.272724,43.31,-55.962724,train,linear_reg,,global
3,2020-07-01,79.77,98.337024,79.77,-18.567024,train,linear_reg,,global
4,2020-08-01,48.27,98.790790,48.27,-50.520790,train,linear_reg,,global
5,2020-09-01,98.01,98.953132,98.01,-0.943132,train,linear_reg,,global
6,2020-10-01,49.77,98.915682,49.77,-49.145682,train,linear_reg,,global
7,2020-11-01,35.48,98.973424,35.48,-63.493424,train,linear_reg,,global
8,2020-12-01,63.98,97.883285,63.98,-33.903285,train,linear_reg,,global
9,2021-01-01,109.76,99.092279,109.76,10.667721,train,linear_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,92.421068,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
1,totaltar,-0.274646,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
2,mean_med_diesel_crack_input1_trade_month_lag2,-0.093034,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
3,mean_nwe_hsfo_crack_trade_month_lag1,-0.470991,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,31.772848,train,linear_reg,,global
1,mae,25.035414,train,linear_reg,,global
2,mape,31.042204,train,linear_reg,,global
3,smape,26.008867,train,linear_reg,,global
4,r_squared,0.18787,train,linear_reg,,global
5,mda,58.536585,train,linear_reg,,global
6,adj_r_squared,0.123755,train,linear_reg,,global
7,rmse,39.604131,test,linear_reg,,global
8,mae,31.329987,test,linear_reg,,global
9,mape,23.042977,test,linear_reg,,global


In [120]:
from py_parsnip import linear_reg

# Create specification with statsmodels engine
spec_lasso  = linear_reg(penalty = 0.1, mixture = 1).set_engine("sklearn")

# Fit model
fit_lasso = spec_lasso.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_lasso = fit_lasso.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_lasso, coefs_lasso, stats_lasso = fit_lasso.extract_outputs()

display(outputs_lasso)
display(coefs_lasso)
display(stats_lasso)

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,103.029775,137.65,34.620225,train,linear_reg,,global
1,2020-05-01,113.53,100.681384,113.53,12.848616,train,linear_reg,,global
2,2020-06-01,43.31,99.273792,43.31,-55.963792,train,linear_reg,,global
3,2020-07-01,79.77,98.339240,79.77,-18.569240,train,linear_reg,,global
4,2020-08-01,48.27,98.793435,48.27,-50.523435,train,linear_reg,,global
5,2020-09-01,98.01,98.955868,98.01,-0.945868,train,linear_reg,,global
6,2020-10-01,49.77,98.918707,49.77,-49.148707,train,linear_reg,,global
7,2020-11-01,35.48,98.976257,35.48,-63.496257,train,linear_reg,,global
8,2020-12-01,63.98,97.887476,63.98,-33.907476,train,linear_reg,,global
9,2021-01-01,109.76,99.094761,109.76,10.665239,train,linear_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,92.428387,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
1,totaltar,-0.274601,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
2,mean_med_diesel_crack_input1_trade_month_lag2,-0.093012,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global
3,mean_nwe_hsfo_crack_trade_month_lag1,-0.470341,NaN,NaN,NaN,NaN,NaN,NaN,linear_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,31.772849,train,linear_reg,,global
1,mae,25.035772,train,linear_reg,,global
2,mape,31.044268,train,linear_reg,,global
3,smape,26.010527,train,linear_reg,,global
4,r_squared,0.18787,train,linear_reg,,global
5,mda,58.536585,train,linear_reg,,global
6,adj_r_squared,0.123755,train,linear_reg,,global
7,rmse,39.60012,test,linear_reg,,global
8,mae,31.327088,test,linear_reg,,global
9,mape,23.040836,test,linear_reg,,global


In [121]:
from py_parsnip import gen_additive_mod

# Create specification with statsmodels engine
spec_gam  = gen_additive_mod()

# Fit model
fit_gam = spec_gam.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_gam = fit_gam.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_gam, coefs_gam, stats_gam = fit_gam.extract_outputs()
display(outputs_gam)
display(coefs_gam)
display(stats_gam)

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,100.416486,137.65,37.233514,train,gen_additive_mod,,global
1,2020-05-01,113.53,88.870157,113.53,24.659843,train,gen_additive_mod,,global
2,2020-06-01,43.31,81.534833,43.31,-38.224833,train,gen_additive_mod,,global
3,2020-07-01,79.77,78.245820,79.77,1.524180,train,gen_additive_mod,,global
4,2020-08-01,48.27,81.788009,48.27,-33.518009,train,gen_additive_mod,,global
5,2020-09-01,98.01,83.113151,98.01,14.896849,train,gen_additive_mod,,global
6,2020-10-01,49.77,83.700077,49.77,-33.930077,train,gen_additive_mod,,global
7,2020-11-01,35.48,83.529688,35.48,-48.049688,train,gen_additive_mod,,global
8,2020-12-01,63.98,80.368959,63.98,-16.388959,train,gen_additive_mod,,global
9,2021-01-01,109.76,83.355294,109.76,26.404706,train,gen_additive_mod,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,gen_additive_mod,,global
1,totaltar,95.202059,NaN,NaN,NaN,NaN,NaN,NaN,gen_additive_mod,,global
2,mean_med_diesel_crack_input1_trade_month_lag2,26.058629,NaN,NaN,NaN,NaN,NaN,NaN,gen_additive_mod,,global
3,mean_nwe_hsfo_crack_trade_month_lag1,36.878530,NaN,NaN,NaN,NaN,NaN,NaN,gen_additive_mod,,global


,metric,value,split,model,model_group_name,group
0,rmse,27.437815,train,gen_additive_mod,,global
1,mae,23.192792,train,gen_additive_mod,,global
2,mape,27.842869,train,gen_additive_mod,,global
3,smape,24.527053,train,gen_additive_mod,,global
4,r_squared,0.394363,train,gen_additive_mod,,global
5,mda,65.853659,train,gen_additive_mod,,global
6,adj_r_squared,0.219982,train,gen_additive_mod,,global
7,rmse,41.489347,test,gen_additive_mod,,global
8,mae,33.758892,test,gen_additive_mod,,global
9,mape,25.374214,test,gen_additive_mod,,global


In [122]:
# from py_parsnip import arima_reg

# # Create specification with statsmodels engine
# spec_auto_arima  = arima_reg().set_engine("auto_arima")

# # Fit model
# fit_auto_arima = spec_auto_arima.fit(train_data, FORMULA_STR)

# # Evaluate on test set
# fit_auto_arima = fit_auto_arima.evaluate(test_data)

# print("Model fitted successfully!")

# # Extract outputs from both models
# outputs_auto_arima, coefs_auto_arima, stats_auto_arima = fit_auto_arima.extract_outputs()
# display(outputs_auto_arima)
# display(coefs_auto_arima)
# display(stats_auto_arima)

In [123]:
from py_parsnip import prophet_reg

# Create specification with statsmodels engine
spec_prophet  = prophet_reg()

# Fit model
fit_prophet = spec_prophet.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_prophet = fit_prophet.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_prophet, coefs_prophet, stats_prophet = fit_prophet.extract_outputs()
display(outputs_prophet)
display(coefs_prophet)
display(stats_prophet)

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,112.812329,137.65,24.837671,train,prophet_reg,,global
1,2020-05-01,113.53,115.589899,113.53,-2.059899,train,prophet_reg,,global
2,2020-06-01,43.31,66.349488,43.31,-23.039488,train,prophet_reg,,global
3,2020-07-01,79.77,70.154397,79.77,9.615603,train,prophet_reg,,global
4,2020-08-01,48.27,64.104879,48.27,-15.834879,train,prophet_reg,,global
5,2020-09-01,98.01,118.531915,98.01,-20.521915,train,prophet_reg,,global
6,2020-10-01,49.77,79.545138,49.77,-29.775138,train,prophet_reg,,global
7,2020-11-01,35.48,58.856986,35.48,-23.376986,train,prophet_reg,,global
8,2020-12-01,63.98,99.834320,63.98,-35.854320,train,prophet_reg,,global
9,2021-01-01,109.76,115.407468,109.76,-5.647468,train,prophet_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,growth,linear,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
1,changepoint_prior_scale,0.05,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
2,seasonality_prior_scale,10.0,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
3,seasonality_mode,additive,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global
4,n_changepoints,25,NaN,NaN,NaN,NaN,NaN,NaN,prophet_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,21.537087,train,prophet_reg,,global
1,mae,17.262224,train,prophet_reg,,global
2,mape,19.910901,train,prophet_reg,,global
3,smape,18.557042,train,prophet_reg,,global
4,r_squared,0.626847,train,prophet_reg,,global
5,mda,80.487805,train,prophet_reg,,global
6,rmse,45.21693,test,prophet_reg,,global
7,mae,38.048444,test,prophet_reg,,global
8,mape,29.854629,test,prophet_reg,,global
9,smape,32.221975,test,prophet_reg,,global


In [124]:
from py_parsnip import exp_smoothing

# Create specification with statsmodels engine
spec_exp_smoothing  = exp_smoothing(error="add", trend="add", season="add", seasonal_period=12)

# Fit model
fit_exp_smoothing = spec_exp_smoothing.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_exp_smoothing = fit_exp_smoothing.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_exp_smoothing, coefs_exp_smoothing, stats_exp_smoothing = fit_exp_smoothing.extract_outputs()
display(outputs_exp_smoothing)
display(coefs_exp_smoothing)
display(stats_exp_smoothing)

Model fitted successfully!


/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,119.568266,137.65,18.081734,train,exp_smoothing,,global
1,2020-05-01,113.53,102.034604,113.53,11.495396,train,exp_smoothing,,global
2,2020-06-01,43.31,72.949596,43.31,-29.639596,train,exp_smoothing,,global
3,2020-07-01,79.77,90.679617,79.77,-10.909617,train,exp_smoothing,,global
4,2020-08-01,48.27,78.521668,48.27,-30.251668,train,exp_smoothing,,global
5,2020-09-01,98.01,69.405629,98.01,28.604371,train,exp_smoothing,,global
6,2020-10-01,49.77,61.577041,49.77,-11.807041,train,exp_smoothing,,global
7,2020-11-01,35.48,89.866576,35.48,-54.386576,train,exp_smoothing,,global
8,2020-12-01,63.98,112.668013,63.98,-48.688013,train,exp_smoothing,,global
9,2021-01-01,109.76,94.222961,109.76,15.537039,train,exp_smoothing,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,alpha (smoothing_level),1.490116e-08,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
1,beta (smoothing_trend),0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
2,gamma (smoothing_seasonal),0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
3,phi (damping_trend),NaN,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
4,l0 (initial_level),8.749568e+01,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global
5,b0 (initial_trend),4.893895e-01,NaN,NaN,NaN,NaN,NaN,NaN,exp_smoothing,,global


,metric,value,split,model,model_group_name,group
0,rmse,29.174087,train,exp_smoothing,,global
1,mae,24.100522,train,exp_smoothing,,global
2,mape,32.149773,train,exp_smoothing,,global
3,smape,26.919124,train,exp_smoothing,,global
4,r_squared,0.315289,train,exp_smoothing,,global
5,mda,68.292683,train,exp_smoothing,,global
6,rmse,26.522722,test,exp_smoothing,,global
7,mae,20.040026,test,exp_smoothing,,global
8,mape,15.517037,test,exp_smoothing,,global
9,smape,17.038668,test,exp_smoothing,,global


In [125]:
from py_parsnip import arima_boost

# Create specification with statsmodels engine
spec_arima_boost  = arima_boost()

# Fit model
fit_arima_boost = spec_arima_boost.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_arima_boost = fit_arima_boost.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_arima_boost, coefs_arima_boost, stats_arima_boost = fit_arima_boost.extract_outputs()
display(outputs_arima_boost)
display(coefs_arima_boost)
display(stats_arima_boost)

/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/matthewdeane/Documents/Data Science/python/_projects/py-tidymodels/py-tidymodels2/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


Model fitted successfully!


,date,actuals,arima_fitted,xgb_fitted,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,69.644426,66.296181,135.940607,137.65,1.709393,train,arima_boost,,global
1,2020-05-01,113.53,53.958858,60.213924,114.172782,113.53,-0.642782,train,arima_boost,,global
2,2020-06-01,43.31,43.831392,1.181154,45.012545,43.31,-1.702545,train,arima_boost,,global
3,2020-07-01,79.77,38.368178,41.388634,79.756812,79.77,0.013188,train,arima_boost,,global
4,2020-08-01,48.27,42.765946,5.786700,48.552646,48.27,-0.282646,train,arima_boost,,global
5,2020-09-01,98.01,44.227059,53.478550,97.705609,98.01,0.304391,train,arima_boost,,global
6,2020-10-01,49.77,44.438479,5.319664,49.758143,49.77,0.011857,train,arima_boost,,global
7,2020-11-01,35.48,44.562212,-7.320201,37.242012,35.48,-1.762012,train,arima_boost,,global
8,2020-12-01,63.98,38.231713,25.720531,63.952244,63.98,0.027756,train,arima_boost,,global
9,2021-01-01,109.76,44.892472,63.695606,108.588078,109.76,1.171922,train,arima_boost,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,arima_totaltar,-0.047637,0.137863,-0.345539,7.296893e-01,NaN,NaN,NaN,arima_boost,,global
1,arima_mean_med_diesel_crack_input1_trade_month...,-0.706213,0.134381,-5.255296,1.477866e-07,NaN,NaN,NaN,arima_boost,,global
2,arima_mean_nwe_hsfo_crack_trade_month_lag1,-2.621992,0.557571,-4.702529,2.569593e-06,NaN,NaN,NaN,arima_boost,,global
3,arima_sigma2,4255.375062,1046.004270,4.068220,4.737373e-05,NaN,NaN,NaN,arima_boost,,global
4,xgb_n_estimators,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,arima_boost,,global
5,xgb_max_depth,6.000000,NaN,NaN,NaN,NaN,NaN,NaN,arima_boost,,global
6,xgb_learning_rate,0.100000,NaN,NaN,NaN,NaN,NaN,NaN,arima_boost,,global
7,xgb_min_child_weight,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,arima_boost,,global
8,xgb_gamma,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,arima_boost,,global
9,xgb_subsample,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,arima_boost,,global


,metric,value,split,model,model_group_name,group
0,rmse,1.043832,train,arima_boost,,global
1,mae,0.756105,train,arima_boost,,global
2,mape,0.939863,train,arima_boost,,global
3,smape,0.931641,train,arima_boost,,global
4,r_squared,0.999123,train,arima_boost,,global
5,mda,97.560976,train,arima_boost,,global
6,rmse,52.801972,test,arima_boost,,global
7,mae,40.741473,test,arima_boost,,global
8,mape,30.73324,test,arima_boost,,global
9,smape,39.25544,test,arima_boost,,global


In [126]:
from py_parsnip import prophet_boost

# Create specification with statsmodels engine
spec_prophet_boost  = prophet_boost(     
      seasonality_yearly=False,
      seasonality_weekly=False,
      seasonality_daily=False)   

# Fit model
fit_prophet_boost = spec_prophet_boost.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_prophet_boost = fit_prophet_boost.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_prophet_boost, coefs_prophet_boost, stats_prophet_boost = fit_prophet_boost.extract_outputs()
display(outputs_prophet_boost)
display(coefs_prophet_boost)
display(stats_prophet_boost)

Model fitted successfully!


,date,actuals,prophet_fitted,xgb_fitted,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,112.553041,24.644632,137.197673,137.65,0.452327,train,prophet_boost,,global
1,2020-05-01,113.53,122.000742,-8.389091,113.611651,113.53,-0.081651,train,prophet_boost,,global
2,2020-06-01,43.31,70.450176,-26.951851,43.498325,43.31,-0.188325,train,prophet_boost,,global
3,2020-07-01,79.77,71.641046,7.949687,79.590733,79.77,0.179267,train,prophet_boost,,global
4,2020-08-01,48.27,59.675524,-11.255198,48.420327,48.27,-0.150327,train,prophet_boost,,global
5,2020-09-01,98.01,109.218005,-11.491523,97.726482,98.01,0.283518,train,prophet_boost,,global
6,2020-10-01,49.77,75.128970,-25.275051,49.853919,49.77,-0.083919,train,prophet_boost,,global
7,2020-11-01,35.48,55.915862,-20.377525,35.538337,35.48,-0.058337,train,prophet_boost,,global
8,2020-12-01,63.98,96.949583,-32.496399,64.453184,63.98,-0.473184,train,prophet_boost,,global
9,2021-01-01,109.76,116.987087,-7.293860,109.693227,109.76,0.066773,train,prophet_boost,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,prophet_growth,linear,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global
1,prophet_changepoint_prior_scale,0.05,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global
2,prophet_seasonality_prior_scale,10.0,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global
3,prophet_seasonality_mode,additive,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global
4,prophet_n_changepoints,25,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global
5,prophet_changepoint_range,0.8,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global
6,prophet_n_changepoints_detected,25,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global
7,xgb_n_estimators,100,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global
8,xgb_max_depth,6,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global
9,xgb_learning_rate,0.1,NaN,NaN,NaN,NaN,NaN,NaN,prophet_boost,,global


,metric,value,split,model,model_group_name,group
0,rmse,0.242193,train,prophet_boost,,global
1,mae,0.189822,train,prophet_boost,,global
2,mape,0.21857,train,prophet_boost,,global
3,smape,0.21829,train,prophet_boost,,global
4,r_squared,0.999953,train,prophet_boost,,global
5,mda,97.560976,train,prophet_boost,,global
6,rmse,39.859288,test,prophet_boost,,global
7,mae,32.367424,test,prophet_boost,,global
8,mape,25.33341,test,prophet_boost,,global
9,smape,28.699086,test,prophet_boost,,global


In [127]:
from py_parsnip import boost_tree   

# Create specification with statsmodels engine
spec_boost_tree  = boost_tree().set_engine("lightgbm")

# Fit model
fit_boost_tree = spec_boost_tree.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_boost_tree = fit_boost_tree.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_boost_tree, coefs_boost_tree, stats_boost_tree = fit_boost_tree.extract_outputs()
display(outputs_boost_tree)
display(coefs_boost_tree)
display(stats_boost_tree)

Model fitted successfully!


,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,96.932889,137.65,40.717111,train,boost_tree,,global
1,2020-05-01,113.53,88.129225,113.53,25.400775,train,boost_tree,,global
2,2020-06-01,43.31,88.129225,43.31,-44.819225,train,boost_tree,,global
3,2020-07-01,79.77,88.129225,79.77,-8.359225,train,boost_tree,,global
4,2020-08-01,48.27,88.129225,48.27,-39.859225,train,boost_tree,,global
5,2020-09-01,98.01,88.129225,98.01,9.880775,train,boost_tree,,global
6,2020-10-01,49.77,88.129225,49.77,-38.359225,train,boost_tree,,global
7,2020-11-01,35.48,88.129225,35.48,-52.649225,train,boost_tree,,global
8,2020-12-01,63.98,88.129225,63.98,-24.149225,train,boost_tree,,global
9,2021-01-01,109.76,88.129225,109.76,21.630775,train,boost_tree,,global


,variable,importance,model,model_group_name,group
0,mean_med_diesel_crack_input1_trade_month_lag2,52,boost_tree,,global
1,mean_nwe_hsfo_crack_trade_month_lag1,48,boost_tree,,global
2,Intercept,0,boost_tree,,global
3,totaltar,0,boost_tree,,global


,metric,value,split,model,model_group_name,group
0,rmse,34.09723,train,boost_tree,,global
1,mae,27.775788,train,boost_tree,,global
2,mape,38.451064,train,boost_tree,,global
3,smape,30.102904,train,boost_tree,,global
4,r_squared,0.064699,train,boost_tree,,global
5,mda,51.219512,train,boost_tree,,global
6,rmse,28.19205,test,boost_tree,,global
7,mae,21.750719,test,boost_tree,,global
8,mape,15.849579,test,boost_tree,,global
9,smape,17.886132,test,boost_tree,,global


In [128]:
from py_parsnip import null_model

# Create specification with statsmodels engine
spec_null_model  = null_model(strategy="mean")

# Fit model
fit_null_model = spec_null_model.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_null_model = fit_null_model.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_null_model, coefs_null_model, stats_null_model = fit_null_model.extract_outputs()
display(outputs_null_model)
display(coefs_null_model)
display(stats_null_model)

Model fitted successfully!


,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,137.65,99.005,99.005,38.645,train,null_model,,global
1,113.53,99.005,99.005,14.525,train,null_model,,global
2,43.31,99.005,99.005,-55.695,train,null_model,,global
3,79.77,99.005,99.005,-19.235,train,null_model,,global
4,48.27,99.005,99.005,-50.735,train,null_model,,global
5,98.01,99.005,99.005,-0.995,train,null_model,,global
6,49.77,99.005,99.005,-49.235,train,null_model,,global
7,35.48,99.005,99.005,-63.525,train,null_model,,global
8,63.98,99.005,99.005,-35.025,train,null_model,,global
9,109.76,99.005,99.005,10.755,train,null_model,,global


,variable,coefficient,std_error,p_value,ci_0.025,ci_0.975,model,model_group_name,group
0,(Intercept),99.005,NaN,NaN,NaN,NaN,null_model,,global


,metric,value,split,model,model_group_name,group
0,rmse,35.256848,train,null_model,,global
1,mae,27.009762,train,null_model,,global
2,mape,38.484528,train,null_model,,global
3,r_squared,0.0,train,null_model,,global
4,baseline_value,99.005,train,null_model,,global
5,rmse,32.591708,test,null_model,,global
6,mae,25.992333,test,null_model,,global
7,mape,19.028925,test,null_model,,global
8,r_squared,-1.747465,test,null_model,,global
9,train_start_date,2020-04-01T00:00:00.000000000,train,null_model,,global


In [129]:
from py_parsnip import naive_reg

# Create specification with statsmodels engine
spec_naive_model  = naive_reg(strategy="naive")

# Fit model
fit_naive_model = spec_naive_model.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_naive_model = fit_naive_model.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_naive_model, coefs_naive_model, stats_naive_model = fit_naive_model.extract_outputs()
display(outputs_naive_model)
display(coefs_naive_model)
display(stats_naive_model)

Model fitted successfully!


,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,137.65,137.65,137.65,0.00,train,naive_reg,,global
1,113.53,137.65,137.65,-24.12,train,naive_reg,,global
2,43.31,113.53,113.53,-70.22,train,naive_reg,,global
3,79.77,43.31,43.31,36.46,train,naive_reg,,global
4,48.27,79.77,79.77,-31.50,train,naive_reg,,global
5,98.01,48.27,48.27,49.74,train,naive_reg,,global
6,49.77,98.01,98.01,-48.24,train,naive_reg,,global
7,35.48,49.77,49.77,-14.29,train,naive_reg,,global
8,63.98,35.48,35.48,28.50,train,naive_reg,,global
9,109.76,63.98,63.98,45.78,train,naive_reg,,global


,variable,coefficient,std_error,p_value,ci_0.025,ci_0.975,model,model_group_name,group
0,strategy,naive,NaN,NaN,NaN,NaN,naive_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,39.867462,train,naive_reg,,global
1,mae,30.887619,train,naive_reg,,global
2,mape,38.600982,train,naive_reg,,global
3,r_squared,-0.278646,train,naive_reg,,global
4,strategy,naive,train,naive_reg,,global
5,rmse,69.468119,test,naive_reg,,global
6,mae,66.627333,test,naive_reg,,global
7,mape,52.262193,test,naive_reg,,global
8,r_squared,-11.482139,test,naive_reg,,global
9,train_start_date,2020-04-01T00:00:00.000000000,train,naive_reg,,global


In [130]:
from py_parsnip import naive_reg

# Create specification with statsmodels engine
spec_naive_model  = naive_reg(strategy="seasonal_naive", seasonal_period=12)

# Fit model
fit_naive_model = spec_naive_model.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_naive_model = fit_naive_model.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_naive_model, coefs_naive_model, stats_naive_model = fit_naive_model.extract_outputs()
display(outputs_naive_model)
display(coefs_naive_model)
display(stats_naive_model)

Model fitted successfully!


,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,137.65,137.65,137.65,0.00,train,naive_reg,,global
1,113.53,113.53,113.53,0.00,train,naive_reg,,global
2,43.31,43.31,43.31,0.00,train,naive_reg,,global
3,79.77,79.77,79.77,0.00,train,naive_reg,,global
4,48.27,48.27,48.27,0.00,train,naive_reg,,global
5,98.01,98.01,98.01,0.00,train,naive_reg,,global
6,49.77,49.77,49.77,0.00,train,naive_reg,,global
7,35.48,35.48,35.48,0.00,train,naive_reg,,global
8,63.98,63.98,63.98,0.00,train,naive_reg,,global
9,109.76,109.76,109.76,0.00,train,naive_reg,,global


,variable,coefficient,std_error,p_value,ci_0.025,ci_0.975,model,model_group_name,group
0,strategy,seasonal_naive,NaN,NaN,NaN,NaN,naive_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,42.541986,train,naive_reg,,global
1,mae,27.645714,train,naive_reg,,global
2,mape,33.286186,train,naive_reg,,global
3,r_squared,-0.455957,train,naive_reg,,global
4,strategy,seasonal_naive,train,naive_reg,,global
5,rmse,44.570802,test,naive_reg,,global
6,mae,31.266,test,naive_reg,,global
7,mape,24.715076,test,naive_reg,,global
8,r_squared,-4.138292,test,naive_reg,,global
9,train_start_date,2020-04-01T00:00:00.000000000,train,naive_reg,,global


In [131]:
from py_parsnip import naive_reg

# Create specification with statsmodels engine
spec_naive_model  = naive_reg(strategy="drift")

# Fit model
fit_naive_model = spec_naive_model.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_naive_model = fit_naive_model.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_naive_model, coefs_naive_model, stats_naive_model = fit_naive_model.extract_outputs()
display(outputs_naive_model)
display(coefs_naive_model)
display(stats_naive_model)

Model fitted successfully!


,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,137.65,137.650000,137.650000,0.000000,train,naive_reg,,global
1,113.53,135.716341,135.716341,-22.186341,train,naive_reg,,global
2,43.31,111.596341,111.596341,-68.286341,train,naive_reg,,global
3,79.77,41.376341,41.376341,38.393659,train,naive_reg,,global
4,48.27,77.836341,77.836341,-29.566341,train,naive_reg,,global
5,98.01,46.336341,46.336341,51.673659,train,naive_reg,,global
6,49.77,96.076341,96.076341,-46.306341,train,naive_reg,,global
7,35.48,47.836341,47.836341,-12.356341,train,naive_reg,,global
8,63.98,33.546341,33.546341,30.433659,train,naive_reg,,global
9,109.76,62.046341,62.046341,47.713659,train,naive_reg,,global


,variable,coefficient,std_error,p_value,ci_0.025,ci_0.975,model,model_group_name,group
0,strategy,drift,NaN,NaN,NaN,NaN,naive_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,39.821659,train,naive_reg,,global
1,mae,30.808722,train,naive_reg,,global
2,mape,38.155328,train,naive_reg,,global
3,r_squared,-0.275709,train,naive_reg,,global
4,strategy,drift,train,naive_reg,,global
5,rmse,84.801318,test,naive_reg,,global
6,mae,82.096602,test,naive_reg,,global
7,mape,64.940391,test,naive_reg,,global
8,r_squared,-17.600436,test,naive_reg,,global
9,train_start_date,2020-04-01T00:00:00.000000000,train,naive_reg,,global


In [132]:
from py_parsnip import null_model

# Create specification with statsmodels engine
spec_null_model  = null_model(strategy="mean")

# Fit model
fit_null_model = spec_null_model.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_null_model = fit_null_model.evaluate(test_data)

print("Model fitted successfully!")

# Extract outputs from both models
outputs_null_model, coefs_null_model, stats_null_model = fit_null_model.extract_outputs()
display(outputs_null_model)
display(coefs_null_model)
display(stats_null_model)

Model fitted successfully!


,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,137.65,99.005,99.005,38.645,train,null_model,,global
1,113.53,99.005,99.005,14.525,train,null_model,,global
2,43.31,99.005,99.005,-55.695,train,null_model,,global
3,79.77,99.005,99.005,-19.235,train,null_model,,global
4,48.27,99.005,99.005,-50.735,train,null_model,,global
5,98.01,99.005,99.005,-0.995,train,null_model,,global
6,49.77,99.005,99.005,-49.235,train,null_model,,global
7,35.48,99.005,99.005,-63.525,train,null_model,,global
8,63.98,99.005,99.005,-35.025,train,null_model,,global
9,109.76,99.005,99.005,10.755,train,null_model,,global


,variable,coefficient,std_error,p_value,ci_0.025,ci_0.975,model,model_group_name,group
0,(Intercept),99.005,NaN,NaN,NaN,NaN,null_model,,global


,metric,value,split,model,model_group_name,group
0,rmse,35.256848,train,null_model,,global
1,mae,27.009762,train,null_model,,global
2,mape,38.484528,train,null_model,,global
3,r_squared,0.0,train,null_model,,global
4,baseline_value,99.005,train,null_model,,global
5,rmse,32.591708,test,null_model,,global
6,mae,25.992333,test,null_model,,global
7,mape,19.028925,test,null_model,,global
8,r_squared,-1.747465,test,null_model,,global
9,train_start_date,2020-04-01T00:00:00.000000000,train,null_model,,global


## Hybrid Model

Combine two models with different strategies (residual, sequential, weighted)

In [133]:
from py_parsnip import hybrid_model, linear_reg, rand_forest

# Create specification - combine linear regression with random forest
# Strategy: 'residual' trains model2 on residuals from model1
spec_hybrid_model = hybrid_model(
    model1=linear_reg(),
    model2=rand_forest().set_mode('regression'),
    strategy='residual'
)

# Fit model
fit_hybrid_model = spec_hybrid_model.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_hybrid_model = fit_hybrid_model.evaluate(test_data)

print("Hybrid model fitted successfully!")

# Extract outputs from hybrid model
outputs_hybrid_model, coefs_hybrid_model, stats_hybrid_model = fit_hybrid_model.extract_outputs()
display(outputs_hybrid_model)
display(coefs_hybrid_model)
display(stats_hybrid_model)

Hybrid model fitted successfully!


,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,137.65,121.965105,137.65,15.684895,train,hybrid_model,,global
1,113.53,107.662008,113.53,5.867992,train,hybrid_model,,global
2,43.31,64.574786,43.31,-21.264786,train,hybrid_model,,global
3,79.77,81.405328,79.77,-1.635328,train,hybrid_model,,global
4,48.27,68.747004,48.27,-20.477004,train,hybrid_model,,global
5,98.01,79.830918,98.01,18.179082,train,hybrid_model,,global
6,49.77,51.930795,49.77,-2.160795,train,hybrid_model,,global
7,35.48,49.438923,35.48,-13.958923,train,hybrid_model,,global
8,63.98,64.076147,63.98,-0.096147,train,hybrid_model,,global
9,109.76,100.198090,109.76,9.561910,train,hybrid_model,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,strategy,residual,NaN,NaN,NaN,NaN,NaN,NaN,hybrid_model,,global
1,model1_type,linear_reg,NaN,NaN,NaN,NaN,NaN,NaN,hybrid_model,,global
2,model2_type,rand_forest,NaN,NaN,NaN,NaN,NaN,NaN,hybrid_model,,global


,metric,value,split,model,model_group_name,group
0,rmse,12.477728,train,hybrid_model,,global
1,mae,9.824556,train,hybrid_model,,global
2,mape,11.77622,train,hybrid_model,,global
3,r_squared,0.874748,train,hybrid_model,,global
4,rmse,41.892818,test,hybrid_model,,global
5,mae,32.902757,test,hybrid_model,,global
6,mape,24.486087,test,hybrid_model,,global
7,r_squared,-3.539386,test,hybrid_model,,global
8,formula,target ~ totaltar +mean_med_diesel_crack_input...,,hybrid_model,,global
9,model_type,hybrid_model,,hybrid_model,,global


## Manual Regression

Specify coefficients manually for comparison with external forecasts or domain knowledge

In [134]:
from py_parsnip import manual_reg

# Create specification with manually specified coefficients
# Useful for comparing with external models or incorporating domain knowledge
spec_manual_reg = manual_reg(
    coefficients={"totaltar": 0.8, "mean_med_diesel_crack_input1_trade_month_lag2": 0.1, "mean_nwe_hsfo_crack_trade_month_lag1": 2},  # Example coefficients
    intercept=0.0
)

# Fit model (validates coefficients and calculates fitted values)
fit_manual_reg = spec_manual_reg.fit(train_data, FORMULA_STR)

# Evaluate on test set
fit_manual_reg = fit_manual_reg.evaluate(test_data)

print("Manual regression model fitted successfully!")

# Extract outputs from manual model
outputs_manual_reg, coefs_manual_reg, stats_manual_reg = fit_manual_reg.extract_outputs()
display(outputs_manual_reg)
display(coefs_manual_reg)
display(stats_manual_reg)

Manual regression model fitted successfully!


,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,137.65,-28.478,137.65,166.128,train,manual_reg,,global
1,113.53,-22.576,113.53,136.106,train,manual_reg,,global
2,43.31,-20.181,43.31,63.491,train,manual_reg,,global
3,79.77,-16.606,79.77,96.376,train,manual_reg,,global
4,48.27,-15.600,48.27,63.870,train,manual_reg,,global
5,98.01,-15.416,98.01,113.426,train,manual_reg,,global
6,49.77,-14.599,49.77,64.369,train,manual_reg,,global
7,35.48,-15.154,35.48,50.634,train,manual_reg,,global
8,63.98,-10.935,63.98,74.915,train,manual_reg,,global
9,109.76,-16.178,109.76,125.938,train,manual_reg,,global


,variable,coefficient,std_error,t_stat,p_value,ci_0.025,ci_0.975,vif,model,model_group_name,group
0,Intercept,0.0,NaN,NaN,NaN,NaN,NaN,NaN,manual_reg,,global
1,totaltar,0.8,NaN,NaN,NaN,NaN,NaN,NaN,manual_reg,,global
2,mean_med_diesel_crack_input1_trade_month_lag2,0.1,NaN,NaN,NaN,NaN,NaN,NaN,manual_reg,,global
3,mean_nwe_hsfo_crack_trade_month_lag1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,manual_reg,,global


,metric,value,split,model,model_group_name,group
0,rmse,134.37408,train,manual_reg,,global
1,mae,128.31,train,manual_reg,,global
2,mape,140.268889,train,manual_reg,,global
3,r_squared,-13.525933,train,manual_reg,,global
4,rmse,121.778772,test,manual_reg,,global
5,mae,118.799133,test,manual_reg,,global
6,mape,96.142107,test,manual_reg,,global
7,r_squared,-37.358455,test,manual_reg,,global
8,formula,target ~ totaltar +mean_med_diesel_crack_input...,,manual_reg,,global
9,model_type,manual_reg,,manual_reg,,global


In [135]:
spec_boost_xgb = boost_tree().set_engine('catboost')

fit_boost = spec_boost_xgb.fit(train_data,FORMULA_STR,original_training_data=train_data)
fit_boost = fit_boost.evaluate(test_data,original_test_data=test_data)

'fit_boost = spec_boost_xgb.fit(train_data,FORMULA_STR)'
'fit_boost = fit_boost.evaluate(test_data)'

outputs_boost, coefs_boost, stats_boost = fit_boost.extract_outputs()

display(outputs_boost)
display(coefs_boost)
display(stats_boost)

,date,actuals,fitted,forecast,residuals,split,model,model_group_name,group
0,2020-04-01,137.65,133.853924,137.65,3.796076,train,boost_tree,,global
1,2020-05-01,113.53,114.106366,113.53,-0.576366,train,boost_tree,,global
2,2020-06-01,43.31,47.679499,43.31,-4.369499,train,boost_tree,,global
3,2020-07-01,79.77,79.962675,79.77,-0.192675,train,boost_tree,,global
4,2020-08-01,48.27,49.670994,48.27,-1.400994,train,boost_tree,,global
5,2020-09-01,98.01,96.263157,98.01,1.746843,train,boost_tree,,global
6,2020-10-01,49.77,48.618400,49.77,1.151600,train,boost_tree,,global
7,2020-11-01,35.48,39.331033,35.48,-3.851033,train,boost_tree,,global
8,2020-12-01,63.98,64.060276,63.98,-0.080276,train,boost_tree,,global
9,2021-01-01,109.76,107.384896,109.76,2.375104,train,boost_tree,,global


,variable,importance,model,model_group_name,group
0,mean_nwe_hsfo_crack_trade_month_lag1,50.227384,boost_tree,,global
1,mean_med_diesel_crack_input1_trade_month_lag2,41.694037,boost_tree,,global
2,totaltar,8.078579,boost_tree,,global
3,Intercept,0.000000,boost_tree,,global


,metric,value,split,model,model_group_name,group
0,rmse,2.333801,train,boost_tree,,global
1,mae,1.814703,train,boost_tree,,global
2,mape,2.194239,train,boost_tree,,global
3,smape,2.159085,train,boost_tree,,global
4,r_squared,0.995618,train,boost_tree,,global
5,mda,97.560976,train,boost_tree,,global
6,rmse,34.32853,test,boost_tree,,global
7,mae,27.419104,test,boost_tree,,global
8,mape,20.606137,test,boost_tree,,global
9,smape,24.020992,test,boost_tree,,global


Recipes

In [136]:
'Create a simple recipe'

rec = (
    recipe()
    .step_normalize()  # Normalize numeric columns
)

# Fit on training data
rec_fit = rec.prep(train_data)

# Apply to training data
train_processed = rec_fit.bake(train_data)
print("\nProcessed training data:")

# Apply to test data (using training parameters!)
test_processed = rec_fit.bake(test_data)
print("\nProcessed test data:")
display(train_processed)
display(test_processed)


Processed training data:

Processed test data:


,date,mean_med_diesel_crack_input1_trade_month_lag2,mean_nwe_hsfo_crack_trade_month_lag1,mean_nwe_lsfo_crack_trade_month,mean_nwe_ulsfo_crack_trade_month lag3,mean_sing_gasoline_vs_vlsfo_trade_month,mean_sing_vlsfo_crack_trade_month_lag3,new_sweet_sr_margin,target,totaltar
0,2020-04-01,-0.088770,-0.094595,0.046119,1.487556,-2.378036,2.031849,0.358746,1.096099,-0.237232
1,2020-05-01,0.172544,0.089438,1.195442,0.793182,-0.956278,1.074411,0.389075,0.411977,-0.237232
2,2020-06-01,0.402280,0.137482,-0.014980,-0.283450,-0.090439,-0.246999,2.220933,-1.579693,-0.237232
3,2020-07-01,0.427807,0.277542,0.105567,-0.597906,0.118757,-0.654823,-0.253895,-0.545568,-0.237232
4,2020-08-01,0.239857,0.358972,0.118778,-1.317998,-0.297698,-0.986637,-0.602676,-1.439011,-0.237232
5,2020-09-01,0.183889,0.378516,0.209601,-0.954445,0.107135,-1.002717,-0.420703,-0.028221,-0.237232
6,2020-10-01,0.141723,0.420859,0.579498,-0.907686,-0.096250,-1.053877,-0.796780,-1.396466,-0.237232
7,2020-11-01,0.161577,0.393987,0.850315,-0.934573,-0.694783,-0.879931,-0.551117,-1.801778,-0.237232
8,2020-12-01,0.187859,0.560105,0.708301,-1.072512,-0.743208,-0.992484,0.216201,-0.993424,-0.237232
9,2021-01-01,0.194855,0.345129,0.657110,-0.839885,-0.747082,-0.700137,0.201036,0.305047,-0.237232


,date,mean_med_diesel_crack_input1_trade_month_lag2,mean_nwe_hsfo_crack_trade_month_lag1,mean_nwe_lsfo_crack_trade_month,mean_nwe_ulsfo_crack_trade_month lag3,mean_sing_gasoline_vs_vlsfo_trade_month,mean_sing_vlsfo_crack_trade_month_lag3,new_sweet_sr_margin,target,totaltar
0,2023-10-01,-0.603078,0.495775,-0.521937,-0.600244,-0.260895,-0.252846,1.486976,0.245201,-0.237232
1,2023-11-01,-0.680981,0.024293,-0.589642,-0.463473,-0.853617,-0.238228,0.813678,0.549255,-0.237232
2,2023-12-01,-0.685519,-0.290842,-0.553312,-0.851575,1.366184,-0.413637,-1.078838,0.397795,-0.237232
3,2024-01-01,-0.589653,-0.052251,-0.350199,-0.721818,0.426739,-0.227996,0.234398,0.670650,-0.237232
4,2024-02-01,-0.479418,1.820648,-0.738261,-0.322026,0.878060,0.545263,1.080571,0.853309,-0.237232
5,2024-03-01,-0.467694,-0.179283,-0.578082,-0.099920,0.568140,0.080430,0.725724,1.320736,0.432258
6,2024-04-01,-0.509860,1.814133,-0.802663,-0.315013,1.211224,-0.156371,0.774250,1.679816,0.983528
7,2024-05-01,-0.574149,-0.285142,-0.829084,0.344293,0.637872,0.179828,0.574080,0.142809,0.050824
8,2024-06-01,-0.677388,-0.042480,-0.814222,0.216874,0.064521,0.138900,-0.763419,2.105265,0.983528
9,2024-07-01,-0.604402,0.142367,-0.612760,0.034513,-0.028455,-0.327394,-0.017331,0.771340,1.244599


## 2.1 - Workflows

In [137]:
# Create a workflow with formula and linear regression
wf_ols_sk = (
    workflow()
    .add_formula(FORMULA_STR)
    .add_model(linear_reg().set_engine("sklearn"))
)

print("Workflow components:")
print(f"  Preprocessor: {wf_ols_sk.preprocessor}")
print(f"  Model: {wf_ols_sk.spec.mode} with {wf_ols_sk.spec.engine} engine")

# Fit the workflow
wf_ols_sk_fit = wf_ols_sk.fit(train_data)

print("Workflow fitted successfully!")
print(f"Type: {type(wf_ols_sk_fit)}")

# Predict on test data
predictions =  wf_ols_sk_fit.predict(test_data)

print(f"Predictions shape: {predictions.shape}")
predictions.head()

# Evaluate on test data
wf_ols_sk_fit = wf_ols_sk_fit.evaluate(test_data)

print("Evaluation complete!")
print(f"Evaluation data keys: {list(wf_ols_sk_fit.fit.evaluation_data.keys())}")

# Extract all outputs
outputs, coefficients, stats = wf_ols_sk_fit.extract_outputs()

print("\n=== OUTPUTS (first 10 rows) ===")
print(outputs.head(10))

print("\n=== COEFFICIENTS ===")
print(coefficients)

print("\n=== STATS ===")
print(stats)

Workflow components:
  Preprocessor: target ~ totaltar +mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1
  Model: regression with sklearn engine
Workflow fitted successfully!
Type: <class 'py_workflows.workflow.WorkflowFit'>
Predictions shape: (15, 1)
Evaluation complete!
Evaluation data keys: ['test_data', 'test_predictions', 'outcome_col', 'original_test_data']

=== OUTPUTS (first 10 rows) ===
        date  actuals      fitted  forecast  residuals  split       model  \
0 2020-04-01   137.65  103.031150    137.65  34.618850  train  linear_reg   
1 2020-05-01   113.53  100.680966    113.53  12.849034  train  linear_reg   
2 2020-06-01    43.31   99.272716     43.31 -55.962716  train  linear_reg   
3 2020-07-01    79.77   98.337003     79.77 -18.567003  train  linear_reg   
4 2020-08-01    48.27   98.790761     48.27 -50.520761  train  linear_reg   
5 2020-09-01    98.01   98.953101     98.01  -0.943101  train  linear_reg   
6 2020-10-01    49.77   98

In [138]:
# method chaining
# Full pipeline in one chain
outputs2, coefficients2, stats2 = (
    workflow()
    .add_formula(FORMULA_STR)
    .add_model(linear_reg().set_engine("sklearn"))
    .fit(train_data)
    .evaluate(test_data)
    .extract_outputs()
)

print("Method chaining complete!")
print(f"\nTest R²: {stats2[(stats2['metric'] == 'r_squared') & (stats2['split'] == 'test')]['value'].values[0]:.4f}")

Method chaining complete!

Test R²: -3.0570


In [139]:
# Extract components
model_fit = wf_ols_sk_fit.extract_fit_parsnip()
preprocessor = wf_ols_sk_fit.extract_preprocessor()
model_spec = wf_ols_sk_fit.extract_spec_parsnip()

print("Extracted components:")
print(f"  Model fit type: {type(model_fit)}")
print(f"  Preprocessor: {preprocessor}")
print(f"  Model spec: {model_spec.mode} with {model_spec.engine} engine")

# Access underlying sklearn model
sklearn_model = model_fit.fit_data["model"]
print(f"\n  Underlying model type: {type(sklearn_model)}")
print(f"  Number of coefficients: {len(sklearn_model.coef_)}")

Extracted components:
  Model fit type: <class 'py_parsnip.model_spec.ModelFit'>
  Preprocessor: target ~ totaltar +mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1
  Model spec: regression with sklearn engine

  Underlying model type: <class 'sklearn.linear_model._base.LinearRegression'>
  Number of coefficients: 3


In [140]:
# Define different formulas (preprocessing strategies)
formulas = [
    "target ~ mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1",                                                        # Minimal model
    "target ~ mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1 + mean_nwe_lsfo_crack_trade_month",                      # Add more features
    "target ~ mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1 + mean_nwe_lsfo_crack_trade_month + totaltar",           # All features
    "target ~ mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1 + mean_nwe_lsfo_crack_trade_month + I(mean_med_diesel_crack_input1_trade_month_lag2*mean_nwe_hsfo_crack_trade_month_lag1)",           # With interaction
    "target ~ mean_med_diesel_crack_input1_trade_month_lag2 + mean_nwe_hsfo_crack_trade_month_lag1 + mean_nwe_lsfo_crack_trade_month + I(mean_med_diesel_crack_input1_trade_month_lag2**2) + I(mean_nwe_hsfo_crack_trade_month_lag1**2)" # With polynomials
]

# Descriptive IDs for each formula
formula_ids = [
    "minimal",
    "medium",
    "full",
    "interaction",
    "polynomial"
]

print(f"Number of preprocessing strategies: {len(formulas)}")

Number of preprocessing strategies: 5
